## import

In [67]:
import numpy as np
import sys,os
import random
import matplotlib.pyplot as plt
import csv
import ast
from collections import OrderedDict

## Input Data 생성

x, y, z 데이터 중 

x가 0~99이면    정답 => [1,0,0]

x가 100~199이면 정답 => [0,1,0]

x가 200~299이면 정답 => [0,0,1]

In [68]:
input_data=np.empty((0,3))
t=np.empty((0,3))
for i in range(5000):
    input_data=np.append(input_data, np.array([[random.randrange(0,300) for j in range(3)]]),axis=0)
    x=input_data[-1,0]
    if x<100:
        t=np.append(t,np.array([[1, 0, 0]]),axis=0)
    elif x<200:
        t=np.append(t,np.array([[0, 1, 0]]),axis=0)
    elif x<300:
        t=np.append(t,np.array([[0, 0, 1]]),axis=0)
print(input_data.shape)

(5000, 3)


추가미션

R, G, B 순서대로 나열 된 데이터 t->(0,0,1),(0,0,1),(0,0,1),(0,0,1).....(0,1,0),(0,1,0),(0,1,0),(0,1,0)....(1,0,0),(1,0,0),(1,0,0),(1,0,0)

In [59]:
input_data=np.empty((0,3))

ax=np.empty((0,3))
bx=np.empty((0,3))
cx=np.empty((0,3))
order_xyz=np.empty((0,3))

at=np.empty((0,3))
bt=np.empty((0,3))
ct=np.empty((0,3))
order_t=np.empty((0,3))
for i in range(3000):
    tmp= np.array([[random.randrange(0,300) for j in range(3)]])
    x=tmp[0,0]
    if x<100:
        at=np.append(at,np.array([[1, 0, 0]]),axis=0)
        ax=np.append(ax,tmp,axis=0)
    elif x<200:
        bt=np.append(bt,np.array([[0, 1, 0]]),axis=0)
        bx=np.append(bx,tmp,axis=0)
    elif x<300:
        ct=np.append(ct,np.array([[0, 0, 1]]),axis=0)
        cx=np.append(cx,tmp,axis=0)
order_t=np.append(order_t,at,axis=0)
order_t=np.append(order_t,bt,axis=0)
order_t=np.append(order_t,ct,axis=0)

order_xyz=np.append(order_xyz,ax,axis=0)
order_xyz=np.append(order_xyz,bx,axis=0)
order_xyz=np.append(order_xyz,cx,axis=0)

ax=np.empty((0,3))
bx=np.empty((0,3))
cx=np.empty((0,3))

at=np.empty((0,3))
bt=np.empty((0,3))
ct=np.empty((0,3))

for i in range(1000):
    tmp= np.array([[random.randrange(0,300) for j in range(3)]])
    x=tmp[0,0]
    if x<100:
        at=np.append(at,np.array([[1, 0, 0]]),axis=0)
        ax=np.append(ax,tmp,axis=0)
    elif x<200:
        bt=np.append(bt,np.array([[0, 1, 0]]),axis=0)
        bx=np.append(bx,tmp,axis=0)
    elif x<300:
        ct=np.append(ct,np.array([[0, 0, 1]]),axis=0)
        cx=np.append(cx,tmp,axis=0)
order_t=np.append(order_t,at,axis=0)
order_t=np.append(order_t,bt,axis=0)
order_t=np.append(order_t,ct,axis=0)

order_xyz=np.append(order_xyz,ax,axis=0)
order_xyz=np.append(order_xyz,bx,axis=0)
order_xyz=np.append(order_xyz,cx,axis=0)



ax=np.empty((0,3))
bx=np.empty((0,3))
cx=np.empty((0,3))

at=np.empty((0,3))
bt=np.empty((0,3))
ct=np.empty((0,3))

for i in range(1000):
    tmp= np.array([[random.randrange(0,300) for j in range(3)]])
    x=tmp[0,0]
    if x<100:
        at=np.append(at,np.array([[1, 0, 0]]),axis=0)
        ax=np.append(ax,tmp,axis=0)
    elif x<200:
        bt=np.append(bt,np.array([[0, 1, 0]]),axis=0)
        bx=np.append(bx,tmp,axis=0)
    elif x<300:
        ct=np.append(ct,np.array([[0, 0, 1]]),axis=0)
        cx=np.append(cx,tmp,axis=0)
order_t=np.append(order_t,at,axis=0)
order_t=np.append(order_t,bt,axis=0)
order_t=np.append(order_t,ct,axis=0)

order_xyz=np.append(order_xyz,ax,axis=0)
order_xyz=np.append(order_xyz,bx,axis=0)
order_xyz=np.append(order_xyz,cx,axis=0)


In [63]:

input_data=order_xyz
t =order_t

# print(order_t[4901,2])
# print(order_t.shape[0])

## Input Data Normalize  (0~1)

정규화 식


<img src = "../simple_network/Data/normalize.png" width = "200" height = "75">



In [69]:
input_max=299
input_min=0
de=input_max-input_min
normalize_input=np.zeros((5000,3),dtype=float)
for i in range(5000):
    for j in range(3):
        normalize_input[i,j]=(input_data[i,j]-input_min)/de
print(input_data)
print(normalize_input)

[[191. 174.  31.]
 [196. 152. 133.]
 [274. 170. 228.]
 ...
 [296.   6. 108.]
 [ 88. 189. 143.]
 [ 14. 109.  26.]]
[[0.63879599 0.5819398  0.10367893]
 [0.65551839 0.5083612  0.44481605]
 [0.91638796 0.56856187 0.76254181]
 ...
 [0.98996656 0.02006689 0.36120401]
 [0.29431438 0.63210702 0.47826087]
 [0.04682274 0.36454849 0.08695652]]


In [70]:
np.savez("../simple_network/Data/input_data_set.npz", normalize_input)
np.savez("../simple_network/Data/t_set.npz", t)


In [98]:
# colum vector로 csv파일에 저장
# np.savetxt("input.csv", normalize_input, delimiter="\n")

np.savetxt("input.csv", normalize_input, delimiter=",")
np.savetxt("ttt.csv", t, delimiter=",")


In [101]:
np.savetxt("no_normal_input.csv", input_data, delimiter="\n")

#### Input Data 파일 읽기 테스트

np.savez()를 사용하여 numpy array 형 변수 읽어오기


In [346]:
read_input=np.load("../simple_network/Data/input_data_set.npz", allow_pickle=True)
read_t=np.load("../simple_network/Data/t_set.npz", allow_pickle=True)
r_input_data=read_input["arr_0"]
r_t_data=read_t["arr_0"]
print(r_input_data)
print(r_t_data)

[[0.65551839 0.56187291 0.17725753]
 [0.64548495 0.76254181 0.32441472]
 [0.78929766 0.93979933 0.22073579]
 ...
 [0.97993311 0.11036789 0.98327759]
 [0.58528428 0.23745819 0.69230769]
 [0.37792642 0.81270903 0.59531773]]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


## 가중치 W, 편향 B  생성 ##
1. 정규분포를 따르는 랜덤수로 생성   (np.random.randn()을 사용하면 "평균 = 0, 표준편차 = 1"인 랜덤수가 생성됨.)
    
    0.01을 곱해주는 이유: w의 초기값이크면 gradient vanishing 문제가 발생할 수 있음.

    따라서 초기 가중치 값에 0.01을 곱해 표준편차를 0.01로 만들어줌.


In [78]:
weight_init_std=0.1
params={}
params['W1']=weight_init_std*np.random.randn(3,4)
params['B1']=np.zeros(4)
params['W2']=weight_init_std*np.random.randn(4,4)
params['B2']=np.zeros(4)
params['W3']=weight_init_std*np.random.randn(4,3)
params['B3']=np.zeros(3)


2. constant로 생성

In [348]:
# params['W1']=np.zeros((3,4))
# params['B1']=np.zeros(4)
# params['W2']=np.zeros((4,4))
# params['B2']=np.zeros(4)
# params['W3']=np.zeros((4,3))
# params['B3']=np.zeros(3)

# params['W1'].fill(0.1)
# params['W2'].fill(0.1)
# params['W3'].fill(0.1)


3. 균일분포를 따르는 랜덤수로 생성

In [349]:
# weight_init_std=0.01
# params={}
# params['W1']=np.random.uniform(-1,1,(3,4))
# params['B1']=np.zeros(4)
# params['W2']=np.random.uniform(-1,1,(4,4))
# params['B2']=np.zeros(4)
# params['W3']=np.random.uniform(-1,1,(4,3))
# params['B3']=np.zeros(3)
# print(params)

## 가중치 W, 편향 B weight_bias.npz 파일에 저장

np.savez() --> 여러개의 배열을 1개의 압축되지 않은 *.npz 포맷 파일로 저장하기 (Save several arrays into a single file in uncompressed .npz format)

In [79]:
np.savez("../simple_network/Data/weight_bias.npz", **params)

##### dictionary형 변수 csv파일에 저장

In [80]:
# open a file in write mode
# with open('../simple_network/Data/weight_bias.csv', 'w', newline='') as f:
#     # create a csv writer
#     writer = csv.writer(f)
#     # write the keys of the dictionary as the header row
#     writer.writerow(params.keys())
#     # write the values of the dictionary as a single row
#     writer.writerow(params.values())

new_params={}
new_params['W1']=params['W1']
new_params['W2']=params['W2']
new_params['W3']=params['W3']
with open('eeeeeeeee.csv', 'w', newline='') as csvfile:
    # Create a CSV writer
    writer = csv.writer(csvfile)

    # Write the values of the W1, W2, and W3 arrays to the CSV file
    for key, value in new_params.items():
        writer.writerows(value)

npz 변수 파일 불러오기

np.load("파일명",allow_pickle=True)

In [352]:
my_dictionary = np.load("../simple_network/Data/weight_bias.npz", allow_pickle=True)
paaa=dict(my_dictionary)
print(paaa)
print(params)

{'W1': array([[ 1.388474  ,  0.7743251 ,  0.93929876,  0.4429259 ],
       [-0.71189431,  1.02840116,  2.15536279, -0.59042978],
       [ 0.29938707, -0.68912052, -0.53406847,  0.5122242 ]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[ 0.60578169, -2.57270916,  1.3943899 , -0.22412641],
       [-0.37827146, -0.43382424,  2.02736543,  0.33435353],
       [ 0.57139648, -0.03689031,  0.78209073, -0.7718887 ],
       [ 0.17709309,  1.05623998,  2.26935956, -1.03287888]]), 'B2': array([0., 0., 0., 0.]), 'W3': array([[ 1.6043027 , -2.06926336, -1.78399435],
       [-1.48888706, -0.57544497,  0.27609352],
       [ 0.25916228, -0.24607088, -0.75507196],
       [ 0.93656552,  0.66641827, -0.19198389]]), 'B3': array([0., 0., 0.])}
{'W1': array([[ 1.388474  ,  0.7743251 ,  0.93929876,  0.4429259 ],
       [-0.71189431,  1.02840116,  2.15536279, -0.59042978],
       [ 0.29938707, -0.68912052, -0.53406847,  0.5122242 ]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[ 0.60578169, -2.57270916,  1.3